In [ ]:
!pip3 install chromadb

In [4]:
import chromadb

import textwrap
import os

In [5]:
client = chromadb.Client()

Failed to send telemetry event ClientStartEvent: module 'chromadb' has no attribute 'get_settings'


In [6]:
collection = client.create_collection(name="demo")

Failed to send telemetry event ClientCreateCollectionEvent: module 'chromadb' has no attribute 'get_settings'


### Download data

In [7]:
data_dir = "./data/"
if not os.path.isdir(data_dir):
    os.system("mkdir "+data_dir)

os.system("wget -nv -O "+data_dir+"train.csv 'https://www.dropbox.com/scl/fi/e98v27l171otn75x8woe3/train_large.csv?rlkey=6irpgini3g8gwcn78zwu253pp&dl=0'")
os.system("wget -nv -O "+data_dir+"test.csv 'https://www.dropbox.com/scl/fi/56r5rrrlm2l7oz9gb7tm9/test_large.csv?rlkey=a0rqjdmhsizxlh0uuyft7vi65&dl=0'")

2024-04-01 15:02:45 URL:https://ucd0c4c275ff93c76e637dfe1d8c.dl.dropboxusercontent.com/cd/0/inline/CQPR1_8pT_xOBoS3fz16hH0V64cSuF0Piy6Y58rZfzUo03j_5r4dWvWyXp3WmeNhnJ4aoYmS1KMPDYyScu0WGJ4EJgAaRcXatZmploX13tBiZ4ued_VzFntYaVSXcshr3Ek7BPkA5UwMTX12StdzYOUw/file [216786183/216786183] -> "./data/train.csv" [1]
2024-04-01 15:02:46 URL:https://uc6f6da7bd7f7b7762bf71dd1bf5.dl.dropboxusercontent.com/cd/0/inline/CQPmWBcEPMXcwPKb0HkRp-_FhPq5TbUuq7xBYNq7MLuWINPFDADx0rq4H1gMa0OQvFgjpcWLtlLZXrGlLYYkgnYF85X-_UUMvNKT0359PGk_RrZGeNn4WsaSQcuWp-VxmhKGyay7gN3OLAV6c1audMTF/file [1510895/1510895] -> "./data/test.csv" [1]


0

In [8]:
train_file = data_dir+"train.csv"
test_file = data_dir+"test.csv"

In [9]:
import pandas as pd
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

In [11]:
train_answers = train_df['answer'].to_list()
answer_ids = train_df['id'].to_list()
print(len(train_answers))

162402


### Add documents

In [12]:
from tqdm import tqdm
total_docs = len(train_answers)
max_batch_size = 512  
for start_idx in tqdm(range(0, total_docs, max_batch_size)):
    end_idx = min(start_idx + max_batch_size, total_docs)
    batch_documents = train_answers[start_idx:end_idx]
    batch_ids = answer_ids[start_idx:end_idx]
    collection.add(documents=batch_documents, ids=list(map(str, batch_ids)))

100%|██████████| 318/318 [1:07:55<00:00, 12.82s/it]


### Evaluate metrics

-   Relevance denotes that model predicts a revalant answer to query
-   Helpfulness denotes that model predicts the highest scoring answer

In [13]:
def precision_at_k(predicted, original, k):
    total_precision = 0
    for pred_instance, orig_instance in zip(predicted, original):
        total_precision += len(set(pred_instance[:k]) & set(orig_instance)) / k
    return total_precision / len(predicted) 
questions = test_df['query'].to_list()
true_labels_all = list(map(lambda x: list(map(int, x.split(","))), test_df['ids'].to_list()))
predicted_all, original_all, original_top  = [], [], []
results = collection.query(query_texts=questions,n_results=5)['ids']
for result, true_labels in list(zip(results, true_labels_all)):
    predicted_all.append(list(map(int, result)))
    original_all.append(true_labels)
    original_top.append(true_labels[:1])
print("Relevance Precision@1 =", precision_at_k(predicted_all, original_all, 1)) 
print("Helpfulness Precision@1 =", precision_at_k(predicted_all, original_top, 1)) 

Failed to send telemetry event CollectionQueryEvent: module 'chromadb' has no attribute 'get_settings'


Relevance Precision@1 = 0.5035
Helpfulness Precision@1 = 0.1605


In [14]:
wrapper = textwrap.TextWrapper(width=100)

In [20]:
query = 'I require something similar, a collection of elements without any duplicates. Does Common Lisp, specifically SBCL, offer anything of this sort?'


results = collection.query(query_texts=[query],n_results=1)
for answer in results['documents'][0]:
    wrapped_text = wrapper.wrap(text = answer)
    for element in wrapped_text:
        print(element)
    print()

 See Common Lisp Implementations: A Survey by Daniel Weinreb.



In [16]:
query = 'I am utilizing a particular command in my C# code, which functions properly. However, it is reported to malfunction in "unmanaged" code. What is managed or unmanaged code?'


results = collection.query(query_texts=[query],n_results=1)
for answer in results['documents'][0]:
    wrapped_text = wrapper.wrap(text = answer)
    for element in wrapped_text:
        print(element)
    print()

 If you already wrote tests for replaceit() and changeit() the test for modify would simply check
that different results are returned depending on the value of 'value'. However you will simply be
reimplementing the logic of the method in the test, which is a little absurd. In this case I
wouldn't test modify until it has more complex logic, or better - is used by another method that is
more significant to test.

